In [1]:
import pandas as pd
import numpy as np
dataset=pd.read_csv("ckd.csv")


In [2]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [3]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]



In [4]:

dependent=dataset['classification_yes']


In [5]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [6]:
from sklearn.preprocessing import StandardScaler
SC=StandardScaler()
x_train=SC.fit_transform(x_train)
x_test=SC.transform(x_test)

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid= {'n_neighbors':[1,5,10,20,50,100], 'weights':['uniform', 'distance'], 'algorithm':['auto','ball_tree','kd_tree','brute']}

grid=GridSearchCV(KNeighborsClassifier(),param_grid,scoring='f1_weighted')
grid.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 5, 10, 20, 50, 100],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted')

In [19]:
# print best parameter after tuning 
#print(grid.best_params_) 


In [20]:
re=grid.cv_results_
print(re)


{'mean_fit_time': array([0.00257225, 0.        , 0.0016006 , 0.00100183, 0.00301943,
       0.00036912, 0.00163255, 0.0013217 , 0.        , 0.00167804,
       0.00011582, 0.00095034, 0.00097961, 0.0033812 , 0.        ,
       0.00196657, 0.00254202, 0.00037761, 0.0016871 , 0.        ,
       0.00326996, 0.00258636, 0.        , 0.00198679, 0.00165544,
       0.00257006, 0.00250492, 0.00167952, 0.00037251, 0.00025821,
       0.00099239, 0.00234728, 0.00050888, 0.00147371, 0.        ,
       0.00064163, 0.00232058, 0.00100121, 0.        , 0.        ,
       0.00040278, 0.00131683, 0.00011559, 0.00280881, 0.0002727 ,
       0.00127325, 0.00036402, 0.00220242]), 'std_fit_time': array([0.00248946, 0.        , 0.0032012 , 0.00200367, 0.00371348,
       0.00073824, 0.00326509, 0.00264339, 0.        , 0.00335608,
       0.00023165, 0.00190067, 0.00127386, 0.00378067, 0.        ,
       0.00364844, 0.003117  , 0.00075521, 0.00328095, 0.        ,
       0.00400489, 0.00362852, 0.        , 0.00313

In [21]:
grid_predictions = grid.predict(x_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
cm


array([[51,  0],
       [ 2, 80]], dtype=int64)

In [22]:

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        51
           1       1.00      0.98      0.99        82

    accuracy                           0.98       133
   macro avg       0.98      0.99      0.98       133
weighted avg       0.99      0.98      0.99       133



In [23]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'algorithm': 'auto', 'n_neighbors': 1, 'weights': 'uniform'}: 0.9850141736106648


In [24]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


0.9878048780487805

In [25]:
table=pd.DataFrame.from_dict(re)

In [26]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002572,0.002489,0.002896,0.003554,auto,1,uniform,"{'algorithm': 'auto', 'n_neighbors': 1, 'weigh...",0.981569,0.962264,0.962573,0.981217,1.000000,0.977525,0.014082,1
1,0.000000,0.000000,0.004908,0.004009,auto,1,distance,"{'algorithm': 'auto', 'n_neighbors': 1, 'weigh...",0.981569,0.962264,0.962573,0.981217,1.000000,0.977525,0.014082,1
2,0.001601,0.003201,0.004655,0.003041,auto,5,uniform,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.963284,0.962636,0.944023,0.962573,0.962573,0.959018,0.007502,13
3,0.001002,0.002004,0.004942,0.003603,auto,5,distance,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.963284,0.981233,0.944023,0.962573,0.962573,0.962737,0.011770,9
4,0.003019,0.003713,0.002402,0.002866,auto,10,uniform,"{'algorithm': 'auto', 'n_neighbors': 10, 'weig...",0.854321,0.925764,0.907035,0.981217,0.962573,0.926182,0.044461,21
5,0.000369,0.000738,0.006038,0.003127,auto,10,distance,"{'algorithm': 'auto', 'n_neighbors': 10, 'weig...",0.908877,0.944161,0.925524,0.981217,0.962573,0.944470,0.025706,17
6,0.001633,0.003265,0.006098,0.003528,auto,20,uniform,"{'algorithm': 'auto', 'n_neighbors': 20, 'weig...",0.835928,0.907402,0.832377,0.925524,0.944023,0.889051,0.046310,29
7,0.001322,0.002643,0.002478,0.003588,auto,20,distance,"{'algorithm': 'auto', 'n_neighbors': 20, 'weig...",0.835928,0.907402,0.869925,0.962573,0.962573,0.907680,0.050200,25
8,0.000000,0.000000,0.007576,0.001358,auto,50,uniform,"{'algorithm': 'auto', 'n_neighbors': 50, 'weig...",0.779609,0.870619,0.692938,0.813342,0.925524,0.816407,0.079314,37
9,0.001678,0.003356,0.003332,0.004081,auto,50,distance,"{'algorithm': 'auto', 'n_neighbors': 50, 'weig...",0.779609,0.889033,0.713954,0.832377,0.925524,0.828100,0.075635,33
